## 0. Setup Paths

In [1]:
import pandas as pd
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'Generate_tfrecord_fromCSV.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('TF_Sign_Detection', 'workspace'),
    'SCRIPTS_PATH': os.path.join('TF_Sign_Detection','scripts'),
    'APIMODEL_PATH': os.path.join('TF_Sign_Detection','models'),
    'ANNOTATION_PATH': os.path.join('TF_Sign_Detection', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('TF_Sign_Detection', 'workspace','images'),
    'MODEL_PATH': os.path.join('TF_Sign_Detection', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('TF_Sign_Detection', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('TF_Sign_Detection', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('TF_Sign_Detection', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('TF_Sign_Detection', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('TF_Sign_Detection', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('TF_Sign_Detection','protoc')
 }
paths

{'WORKSPACE_PATH': 'TF_Sign_Detection/workspace',
 'SCRIPTS_PATH': 'TF_Sign_Detection/scripts',
 'APIMODEL_PATH': 'TF_Sign_Detection/models',
 'ANNOTATION_PATH': 'TF_Sign_Detection/workspace/annotations',
 'IMAGE_PATH': 'TF_Sign_Detection/workspace/images',
 'MODEL_PATH': 'TF_Sign_Detection/workspace/models',
 'PRETRAINED_MODEL_PATH': 'TF_Sign_Detection/workspace/pre-trained-models',
 'CHECKPOINT_PATH': 'TF_Sign_Detection/workspace/models/my_ssd_mobnet',
 'OUTPUT_PATH': 'TF_Sign_Detection/workspace/models/my_ssd_mobnet/export',
 'TFJS_PATH': 'TF_Sign_Detection/workspace/models/my_ssd_mobnet/tfjsexport',
 'TFLITE_PATH': 'TF_Sign_Detection/workspace/models/my_ssd_mobnet/tfliteexport',
 'PROTOC_PATH': 'TF_Sign_Detection/protoc'}

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('TF_Sign_Detection', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}
files

{'PIPELINE_CONFIG': 'TF_Sign_Detection/workspace/models/my_ssd_mobnet/pipeline.config',
 'TF_RECORD_SCRIPT': 'TF_Sign_Detection/scripts/Generate_tfrecord_fromCSV.py',
 'LABELMAP': 'TF_Sign_Detection/workspace/annotations/label_map.pbtxt'}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [6]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'TF_Sign_Detection/models'...
remote: Enumerating objects: 70422, done.
remote: Total 70422 (delta 0), reused 0 (delta 0), pack-reused 70422
Receiving objects: 100% (70422/70422), 578.38 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (49679/49679), done.
Updating files: 100% (3242/3242), done.


In [7]:
if os.name=='posix':  
    !brew install protobuf-c
    !cd TF_Sign_Detection/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd TF_Sign_Detection/models/research/slim && pip install -e . 

Running `brew update --preinstall`...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
epinio
==> Updated Formulae
Updated 17 formulae.
==> Updated Casks
Updated 5 casks.

To reinstall 1.4.0_1, run:
  brew reinstall protobuf-c
Processing /Users/wala/Desktop/Kaggle_compet/Traffic_Sign_Detection/TF_Sign_Detection/models/research
  Preparing metadata (setup.py) ... done


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1653456 sha256=f8a6c6a909ac9b05b064f8c5b88b277fca266e88abc6c77ebd357b3ddd2ff540
  Stored in directory: /private/var/folders/c0/jhyh58254rx8htlc0km9zjs80000gp/T/pip-ephem-wheel-cache-j88w13sr/wheels/41/8f/8c/a1300a9fd9952f10da5a143fc060e4a1f3b87047b7d9811dbe
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [8]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.7.4: /Users/wala/Desktop/Kaggle_compet/Traffic_Sign_Detection/TFSD/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-03-21 10:27:29.795812: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/wala/Desktop/Kaggle_compet/Traffic_Sign_Detection/TFSD/lib/python3.7/site-packages/object_detection/builders/model_builder.py:1100: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arch.'
W0321 10:27:30.056422 4385451520 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.

I0321 10:27:34.276934 4385451520 efficientnet_model.py:144] round_filter input=24 output=24
I0321 10:27:34.277061 4385451520 efficientnet_model.py:144] round_filter input=40 output=40
I0321 10:27:34.420578 4385451520 efficientnet_model.py:144] round_filter input=40 output=40
I0321 10:27:34.420705 4385451520 efficientnet_model.py:144] round_filter input=80 output=80
I0321 10:27:34.611940 4385451520 efficientnet_model.py:144] round_filter input=80 output=80
I0321 10:27:34.612061 4385451520 efficientnet_model.py:144] round_filter input=112 output=112
I0321 10:27:34.804876 4385451520 efficientnet_model.py:144] round_filter input=112 output=112
I0321 10:27:34.804997 4385451520 efficientnet_model.py:144] round_filter input=192 output=192
I0321 10:27:35.082650 4385451520 efficientnet_model.py:144] round_filter input=192 output=192
I0321 10:27:35.082778 4385451520 efficientnet_model.py:144] round_filter input=320 output=320
I0321 10:27:35.160421 4385451520 efficientnet_model.py:144] round_filt

I0321 10:27:38.663618 4385451520 efficientnet_model.py:144] round_filter input=1280 output=1408
I0321 10:27:38.703455 4385451520 efficientnet_model.py:454] Building model efficientnet with params ModelConfig(width_coefficient=1.1, depth_coefficient=1.2, resolution=260, dropout_rate=0.3, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=24, output_filters=40, kernel_size=5, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=40, output_filters=80, kernel_size=3, num_repeat=3, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv

I0321 10:27:43.291623 4385451520 ssd_efficientnet_bifpn_feature_extractor.py:146] EfficientDet EfficientNet backbone version: efficientnet-b5
I0321 10:27:43.291747 4385451520 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 288
I0321 10:27:43.291800 4385451520 ssd_efficientnet_bifpn_feature_extractor.py:149] EfficientDet BiFPN num iterations: 7
I0321 10:27:43.293241 4385451520 efficientnet_model.py:144] round_filter input=32 output=48
I0321 10:27:43.305978 4385451520 efficientnet_model.py:144] round_filter input=32 output=48
I0321 10:27:43.306084 4385451520 efficientnet_model.py:144] round_filter input=16 output=24
I0321 10:27:43.453414 4385451520 efficientnet_model.py:144] round_filter input=16 output=24
I0321 10:27:43.453540 4385451520 efficientnet_model.py:144] round_filter input=24 output=40
I0321 10:27:43.980006 4385451520 efficientnet_model.py:144] round_filter input=24 output=40
I0321 10:27:43.980159 4385451520 efficientnet_model.py:144] round_fil

I0321 10:27:50.877171 4385451520 efficientnet_model.py:144] round_filter input=24 output=48
I0321 10:27:50.877327 4385451520 efficientnet_model.py:144] round_filter input=40 output=80
I0321 10:27:51.349400 4385451520 efficientnet_model.py:144] round_filter input=40 output=80
I0321 10:27:51.349558 4385451520 efficientnet_model.py:144] round_filter input=80 output=160
I0321 10:27:52.054419 4385451520 efficientnet_model.py:144] round_filter input=80 output=160
I0321 10:27:52.054573 4385451520 efficientnet_model.py:144] round_filter input=112 output=224
I0321 10:27:52.798913 4385451520 efficientnet_model.py:144] round_filter input=112 output=224
I0321 10:27:52.799058 4385451520 efficientnet_model.py:144] round_filter input=192 output=384
I0321 10:27:54.179843 4385451520 efficientnet_model.py:144] round_filter input=192 output=384
I0321 10:27:54.180001 4385451520 efficientnet_model.py:144] round_filter input=320 output=640
I0321 10:27:54.672474 4385451520 efficientnet_model.py:144] round_fi

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

In [9]:
import object_detection

In [10]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-03-21 11:05:34--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Résolution de download.tensorflow.org (download.tensorflow.org)… 172.217.22.144
Connexion à download.tensorflow.org (download.tensorflow.org)|172.217.22.144|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 20515344 (20M) [application/x-tar]
Sauvegarde en : « ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz »

ssd_mobilenet_v2_fp 100%[===================>]  19,56M  4,41MB/s    ds 10s     

2022-03-21 11:05:45 (1,96 MB/s) — « ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz » sauvegardé [20515344/20515344]

x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coc

# 2. Create Label Map

In [11]:
# Label Overview
labels = [  {'name': 'Speed limit (20km/h)','id':1},
            {'name':'Speed limit (30km/h)', 'id':2},
            {'name':'Speed limit (50km/h)', 'id':3},
            {'name':'Speed limit (60km/h)', 'id':4},
            {'name':'Speed limit (70km/h)', 'id':5},
            {'name':'Speed limit (80km/h)', 'id':6},
            {'name':'End of speed limit (80km/h)', 'id':7},
            {'name':'Speed limit (100km/h)', 'id':8},
            {'name':'Speed limit (120km/h)', 'id':9},
            {'name':'No passing', 'id':10},
            {'name':'No passing veh over 3.5 tons', 'id':11},
            {'name':'Right-of-way at intersection', 'id':12},
            {'name':'Priority road', 'id':13},
            {'name':'Yield', 'id':14},
            {'name':'Stop', 'id':15},
            {'name':'No vehicles', 'id':16},
            {'name':'Veh > 3.5 tons prohibited', 'id':17},
            {'name':'No entry', 'id':18},
            {'name':'General caution', 'id':19},
            {'name':'Dangerous curve left', 'id':20},
            {'name':'Dangerous curve right', 'id':21},
            {'name':'Double curve',  'id':22},
            {'name':'Bumpy road',  'id':23},
            {'name':'Slippery road',  'id':24},
            {'name':'Road narrows on the right',  'id':25},
            {'name':'Road work',  'id':26},
            {'name':'Traffic signals',  'id':27},
            {'name':'Pedestrians',  'id':28},
            {'name':'Children crossing',  'id':29},
            {'name':'Bicycles crossing',  'id':30},
            {'name':'Beware of ice/snow', 'id':31},
            {'name':'Wild animals crossing',  'id':32},
            {'name':'End speed + passing limits',  'id':33},
            {'name':'Turn right ahead',  'id':34},
            {'name':'Turn left ahead',  'id':35},
            {'name':'Ahead only',  'id':36},
            {'name':'Go straight or right',  'id':37},
            {'name':'Go straight or left',  'id':38},
            {'name':'Keep right',  'id':39},
            {'name':'Keep left',  'id':40},
            {'name':'Roundabout mandatory',  'id':41},
            {'name':'End of no passing',  'id':42},
            {'name':'End no passing veh > 3.5 tons' , 'id':43}
]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [12]:
{files['TF_RECORD_SCRIPT']}

{'TF_Sign_Detection/scripts/Generate_tfrecord_fromCSV.py'}

In [13]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'Train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} -i {os.path.join(paths['IMAGE_PATH'])}

Successfully created the TFRecord file: TF_Sign_Detection/workspace/annotations/train.record


In [14]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'Test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} -i {os.path.join(paths['IMAGE_PATH'])}

Successfully created the TFRecord file: TF_Sign_Detection/workspace/annotations/test.record


# 4. Copy Model Config to Training Folder

In [15]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [16]:
os.path.join(paths['PRETRAINED_MODEL_PATH'])

'TF_Sign_Detection/workspace/pre-trained-models'

# 5. Update Config For Transfer Learning

In [17]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [18]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [19]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [20]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [21]:
pipeline_config

model {
  ssd {
    num_classes: 90
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0


In [23]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 320
pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 320
pipeline_config.train_config.batch_size =4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [24]:
pipeline_config.eval_input_reader[0].label_map_path

'TF_Sign_Detection/workspace/annotations/label_map.pbtxt'

In [25]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

In [26]:
pip install opencv-python-headless==4.5.2.52

Note: you may need to restart the kernel to use updated packages.


In [27]:
import cv2

# 6. Train the model

In [28]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [29]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=4000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [30]:
TRAINING_SCRIPT

'TF_Sign_Detection/models/research/object_detection/model_main_tf2.py'

In [31]:
paths['CHECKPOINT_PATH']

'TF_Sign_Detection/workspace/models/my_ssd_mobnet'

In [32]:
files['PIPELINE_CONFIG']

'TF_Sign_Detection/workspace/models/my_ssd_mobnet/pipeline.config'

In [33]:
print(command)

python TF_Sign_Detection/models/research/object_detection/model_main_tf2.py --model_dir=TF_Sign_Detection/workspace/models/my_ssd_mobnet --pipeline_config_path=TF_Sign_Detection/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=4000


In [34]:
!{command}

2022-03-21 11:24:45.082906: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0321 11:24:45.090091 4628336128 cross_device_ops.py:1386] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0321 11:24:45.201093 4628336128 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 4000
I0321 11:24:45.208651 4628336128 config_util.py:552] Maybe overwriting train_steps: 4000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0321 11:24:45.208824 4628336128 config_util.py:552] Maybe overwriting use_b

INFO:tensorflow:Step 200 per-step time 1.038s
I0321 11:29:09.641758 4628336128 model_lib_v2.py:707] Step 200 per-step time 1.038s
INFO:tensorflow:{'Loss/classification_loss': 0.013031347,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.15274061,
 'Loss/total_loss': 0.16577196,
 'learning_rate': 0.0373328}
I0321 11:29:09.642054 4628336128 model_lib_v2.py:708] {'Loss/classification_loss': 0.013031347,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.15274061,
 'Loss/total_loss': 0.16577196,
 'learning_rate': 0.0373328}
INFO:tensorflow:Step 300 per-step time 1.108s
I0321 11:31:00.465830 4628336128 model_lib_v2.py:707] Step 300 per-step time 1.108s
INFO:tensorflow:{'Loss/classification_loss': 0.00557354,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.15226151,
 'Loss/total_loss': 0.15783505,
 'learning_rate': 0.0426662}
I0321 11:31:00.466061 4628336128 model_lib_v2.py:708] {'Loss/classification_loss': 0.00557354,
 'Loss/localization_loss': 0.0,


INFO:tensorflow:Step 1700 per-step time 1.335s
I0321 13:10:18.073889 4628336128 model_lib_v2.py:707] Step 1700 per-step time 1.335s
INFO:tensorflow:{'Loss/classification_loss': 0.00030623513,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.14072633,
 'Loss/total_loss': 0.14103256,
 'learning_rate': 0.07995972}
I0321 13:10:18.074170 4628336128 model_lib_v2.py:708] {'Loss/classification_loss': 0.00030623513,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.14072633,
 'Loss/total_loss': 0.14103256,
 'learning_rate': 0.07995972}
INFO:tensorflow:Step 1800 per-step time 1.357s
I0321 13:12:33.753960 4628336128 model_lib_v2.py:707] Step 1800 per-step time 1.357s
INFO:tensorflow:{'Loss/classification_loss': 0.00035915826,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.1398291,
 'Loss/total_loss': 0.14018826,
 'learning_rate': 0.0799474}
I0321 13:12:33.754245 4628336128 model_lib_v2.py:708] {'Loss/classification_loss': 0.00035915826,
 'Loss/localizati

INFO:tensorflow:Step 3200 per-step time 1.758s
I0321 13:50:12.642971 4628336128 model_lib_v2.py:707] Step 3200 per-step time 1.758s
INFO:tensorflow:{'Loss/classification_loss': 0.0001333377,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.1278728,
 'Loss/total_loss': 0.12800613,
 'learning_rate': 0.07960275}
I0321 13:50:12.643305 4628336128 model_lib_v2.py:708] {'Loss/classification_loss': 0.0001333377,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.1278728,
 'Loss/total_loss': 0.12800613,
 'learning_rate': 0.07960275}
INFO:tensorflow:Step 3300 per-step time 1.766s
I0321 13:53:09.209317 4628336128 model_lib_v2.py:707] Step 3300 per-step time 1.766s
INFO:tensorflow:{'Loss/classification_loss': 0.00013740658,
 'Loss/localization_loss': 0.0,
 'Loss/regularization_loss': 0.12706114,
 'Loss/total_loss': 0.12719855,
 'learning_rate': 0.07956588}
I0321 13:53:09.209671 4628336128 model_lib_v2.py:708] {'Loss/classification_loss': 0.00013740658,
 'Loss/localization

# 7. Evaluate the Model

In [35]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)

python TF_Sign_Detection/models/research/object_detection/model_main_tf2.py --model_dir=TF_Sign_Detection/workspace/models/my_ssd_mobnet --pipeline_config_path=TF_Sign_Detection/workspace/models/my_ssd_mobnet/pipeline.config --checkpoint_dir=TF_Sign_Detection/workspace/models/my_ssd_mobnet


In [ ]:
!{command}

W0322 15:20:28.811857 4589100544 model_lib_v2.py:1090] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0322 15:20:28.812102 4589100544 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0322 15:20:28.812208 4589100544 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0322 15:20:28.812309 4589100544 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0322 15:20:28.812440 4589100544 model_lib_v2.py:1111] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2022-03-22 15:20:28.823399: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical 

INFO:tensorflow:Finished eval step 100
I0322 16:45:25.759294 4589100544 model_lib_v2.py:966] Finished eval step 100
INFO:tensorflow:Finished eval step 200
I0322 16:46:10.986215 4589100544 model_lib_v2.py:966] Finished eval step 200
INFO:tensorflow:Finished eval step 300
I0322 16:46:33.773626 4589100544 model_lib_v2.py:966] Finished eval step 300
INFO:tensorflow:Finished eval step 400
I0322 16:47:11.665417 4589100544 model_lib_v2.py:966] Finished eval step 400
INFO:tensorflow:Finished eval step 500
I0322 16:47:33.952613 4589100544 model_lib_v2.py:966] Finished eval step 500
INFO:tensorflow:Finished eval step 600
I0322 17:42:36.504214 4589100544 model_lib_v2.py:966] Finished eval step 600
INFO:tensorflow:Finished eval step 700
I0322 17:43:01.861690 4589100544 model_lib_v2.py:966] Finished eval step 700
INFO:tensorflow:Finished eval step 800
I0322 18:16:57.820321 4589100544 model_lib_v2.py:966] Finished eval step 800
INFO:tensorflow:Finished eval step 900
I0322 19:38:03.111422 4589100544 

INFO:tensorflow:Finished eval step 7100
I0322 20:49:37.428842 4589100544 model_lib_v2.py:966] Finished eval step 7100
INFO:tensorflow:Finished eval step 7200
I0322 20:49:50.481313 4589100544 model_lib_v2.py:966] Finished eval step 7200
INFO:tensorflow:Finished eval step 7300
I0322 20:50:03.451206 4589100544 model_lib_v2.py:966] Finished eval step 7300
INFO:tensorflow:Finished eval step 7400
I0322 20:50:16.331636 4589100544 model_lib_v2.py:966] Finished eval step 7400
INFO:tensorflow:Finished eval step 7500
I0322 20:50:29.152631 4589100544 model_lib_v2.py:966] Finished eval step 7500
INFO:tensorflow:Finished eval step 7600
I0322 20:50:42.139630 4589100544 model_lib_v2.py:966] Finished eval step 7600
INFO:tensorflow:Finished eval step 7700
I0322 20:50:54.934458 4589100544 model_lib_v2.py:966] Finished eval step 7700
INFO:tensorflow:Finished eval step 7800
I0322 20:51:07.745824 4589100544 model_lib_v2.py:966] Finished eval step 7800
INFO:tensorflow:Finished eval step 7900
I0322 20:51:20.7

INFO:tensorflow:Eval metrics at step 4000
I0322 21:11:24.233849 4589100544 model_lib_v2.py:1015] Eval metrics at step 4000
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP: -1.000000
I0322 21:11:24.345637 4589100544 model_lib_v2.py:1018] 	+ DetectionBoxes_Precision/mAP: -1.000000
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP@.50IOU: -1.000000
I0322 21:11:24.347241 4589100544 model_lib_v2.py:1018] 	+ DetectionBoxes_Precision/mAP@.50IOU: -1.000000
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP@.75IOU: -1.000000
I0322 21:11:24.348316 4589100544 model_lib_v2.py:1018] 	+ DetectionBoxes_Precision/mAP@.75IOU: -1.000000
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP (small): -1.000000
I0322 21:11:24.349147 4589100544 model_lib_v2.py:1018] 	+ DetectionBoxes_Precision/mAP (small): -1.000000
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP (medium): -1.000000
I0322 21:11:24.349868 4589100544 model_lib_v2.py:1018] 	+ DetectionBoxes_Precision/mAP (medium): -1.000000
INFO:tensorflow:	+ DetectionBo

# 8. Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-5')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


# 9. Detect from an Image

In [ ]:

import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline



In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'Train/2', '00002_00002_00011.png')

In [ ]:
IMAGE_PATH

In [ ]:
category_index

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
detections['detection_classes']